In [1]:
from util import *
from glob import glob

In [2]:
df = load_AOIs()
df

,Taranaki,AOI,SSP 4.5 (p50),SSP 4.5 (p83),SSP 8.5 (p50),SSP 8.5 (p83),Rate SSP 4.5 (p50),Rate SSP 4.5 (p83),Rate SSP 8.5 (p50),Rate SSP 8.5 (p83),match,match_score
7,NORTH,TongaporutuRiver,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,TongapurutuRiverCliffs,93.750000
11,SOUTH,HangatahuaRiver_South,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,HangatahuRiver_South,97.435897
21,SOUTH,Rahotu,0.58,0.78,0.84,1.10,0.0058,0.0078,0.0084,0.0110,Rahotu,100.000000
20,SOUTH,Pihama,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,Pihama,100.000000
19,SOUTH,OpunakeBeach,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,OpunakeBeachCliffs,100.000000
18,SOUTH,OhaweBeach,0.57,0.78,0.83,1.10,0.0057,0.0078,0.0083,0.0110,OhaweBeach,100.000000
17,SOUTH,Oeo,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,Oeo,100.000000
16,SOUTH,Manutahi,0.57,0.78,0.83,1.10,0.0057,0.0078,0.0083,0.0110,Manutahi,100.000000
15,SOUTH,ManaBay,0.57,0.78,0.83,1.10,0.0057,0.0078,0.0083,0.0110,ManaBayCliffs,100.000000
14,SOUTH,KaupokonuiBeach,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,KaupokonuiBeach,100.000000


In [3]:
site = df.match.sample(1).iloc[0]
site

'TongapurutuRiverCliffs'

In [4]:
gdf = gpd.read_file(f"Shapefiles/{site}_intersects.shp")
gdf.crs = 2193
gdf = enrich_df(gdf)
transect_metadata = get_transect_metadata(f"Shapefiles/{site}_TransectLines.shp")
if site == "ManaBayCliffs":
  print("Flipping")
  for k, v in transect_metadata.items():
    transect_metadata[k]["Azimuth"] = v["Azimuth"] + 180

In [5]:
linear_models = fit(gdf, transect_metadata)
linear_models.loc[linear_models.slope > 0, "slope"] = pd.NA
rolled_slopes = linear_models.groupby("group").slope.rolling(10, min_periods=1).mean().dropna().reset_index(level=0)
linear_models.slope = rolled_slopes.slope
linear_models.dropna(inplace=True)
linear_models.index = linear_models.TransectID
linear_models

,TransectID,slope,intercept,group
TransectID,,,,
2,2,-0.065018,17.628984,0
3,3,-0.142436,28.436840,0
4,4,-0.151714,24.779193,0
5,5,-0.149376,22.298158,0
6,6,-0.146611,22.375056,0
...,...,...,...,...
713,713,-0.057338,20.215503,31
714,714,-0.073358,21.902249,31
715,715,-0.088760,24.895374,31


In [6]:
# Find the closest data point to 2000
gdf["years_since_2000"] = (gdf.Year - 2000).abs()
closest_to_2000 = gdf.loc[gdf.groupby("TransectID").years_since_2000.idxmin()]
closest_to_2000.set_index("TransectID", inplace=True)
closest_to_2000

,TransOrder,BaselineID,ShorelineI,Distance,IntersectX,IntersectY,Uncertaint,geometry,Date,Year,YearsSinceBase,YearsUntilFuture,years_since_2000
TransectID,,,,,,,,,,,,,
2,2,1,18/04/2011,13.555636,1.735114e+06,5.697807e+06,3.0,POINT (1735113.825 5697806.787),2011-04-18,2011,71.293634,88.706366,11
3,3,1,18/04/2011,13.558883,1.735123e+06,5.697807e+06,3.0,POINT (1735123.435 5697806.987),2011-04-18,2011,71.293634,88.706366,11
4,4,1,18/04/2011,13.712590,1.735133e+06,5.697808e+06,3.0,POINT (1735133.042 5697808.360),2011-04-18,2011,71.293634,88.706366,11
5,5,1,18/04/2011,13.586587,1.735143e+06,5.697811e+06,3.0,POINT (1735142.651 5697810.828),2011-04-18,2011,71.293634,88.706366,11
6,6,1,18/04/2011,14.291941,1.735152e+06,5.697814e+06,3.0,POINT (1735151.779 5697814.147),2011-04-18,2011,71.293634,88.706366,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,725,1,18/04/2011,4.111431,1.738124e+06,5.704176e+06,3.0,POINT (1738123.783 5704175.785),2011-04-18,2011,71.293634,88.706366,11
726,726,1,18/04/2011,4.606597,1.738133e+06,5.704172e+06,3.0,POINT (1738133.054 5704171.749),2011-04-18,2011,71.293634,88.706366,11
727,727,1,18/04/2011,5.187253,1.738142e+06,5.704168e+06,3.0,POINT (1738142.338 5704167.688),2011-04-18,2011,71.293634,88.706366,11


In [7]:
closest_to_2000.Year.value_counts()

2011    661
1970     12
2017     12
Name: Year, dtype: int64

In [8]:
# Predict to the closest year to 2000
linear_models["FUTURE_YEAR"] = closest_to_2000.Year
linear_models

,TransectID,slope,intercept,group,FUTURE_YEAR
TransectID,,,,,
2,2,-0.065018,17.628984,0,2011
3,3,-0.142436,28.436840,0,2011
4,4,-0.151714,24.779193,0,2011
5,5,-0.149376,22.298158,0,2011
6,6,-0.146611,22.375056,0,2011
...,...,...,...,...,...
713,713,-0.057338,20.215503,31,2011
714,714,-0.073358,21.902249,31,2011
715,715,-0.088760,24.895374,31,2011


In [9]:
results = predict(gdf, linear_models, transect_metadata)
results

,TransectID,BaselineID,group,Year,ocean_point,linear_model_point,linear_model_distance,sqrt_model_point,sqrt_model_distance,BH_model_point,BH_model_distance,Sunamura_model_point,Sunamura_model_distance
0,2.0,1,0.0,2011,POINT (1734901.294958501 5698259.049972313),POINT (1735114.055735204 5697806.2951233275),0.254037,POINT (1735116.482537082 5697801.130889888),5.960058,POINT (1735120.7613459069 5697792.025586394),16.020612,POINT (1735116.1250134942 5697801.891699942),5.119430
1,3.0,1,0.0,2011,POINT (1734897.1336462596 5698251.896706724),POINT (1735123.817820887 5697806.235080629),0.000000,POINT (1735126.9420696187 5697800.092800197),6.891193,POINT (1735128.422893738 5697797.181496459),10.157464,POINT (1735123.817820887 5697806.235080629),0.000000
2,4.0,1,0.0,2011,POINT (1734900.895041738 5698249.22434408),POINT (1735133.8567471951 5697806.811817369),0.000000,POINT (1735139.1077513464 5697796.839749171),11.270102,POINT (1735140.2502180436 5697794.670115423),13.722150,POINT (1735135.1687063214 5697804.3203042215),2.815826
3,5.0,1,0.0,2011,POINT (1734907.5357417199 5698248.764666272),POINT (1735144.04181646 5697808.236714139),0.000000,POINT (1735149.7475224163 5697797.608983061),12.062493,POINT (1735151.0044365593 5697795.267792401),14.719749,POINT (1735145.845678354 5697804.876751453),3.813564
4,6.0,1,0.0,2011,POINT (1734912.0278775676 5698248.993504751),POINT (1735153.4398400048 5697811.134778228),0.000000,POINT (1735159.1144658017 5697800.8424778795),11.752992,POINT (1735160.514576642 5697798.30303972),14.652829,POINT (1735155.2488885478 5697807.853633071),3.746808
...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,713.0,1,31.0,2011,POINT (1737658.0284376887 5704476.621755098),POINT (1738025.8406172907 5704137.926906398),0.000000,POINT (1738027.7594203593 5704136.160002659),2.608401,POINT (1738035.6560237126 5704128.88852274),13.342960,POINT (1738027.6371835656 5704136.272562751),2.442233
566,714.0,1,31.0,2011,POINT (1737665.3202893392 5704491.54016579),POINT (1738027.7069854382 5704147.046504759),0.000000,POINT (1738029.1507338744 5704145.674042167),1.992000,POINT (1738035.9118355655 5704139.2467734525),11.320573,POINT (1738028.0105901905 5704146.757890652),0.418896
567,715.0,1,31.0,2011,POINT (1737676.0880781382 5704512.156475647),POINT (1738028.590900001 5704157.555628808),0.000000,POINT (1738029.9165793657 5704156.22205927),1.880381,POINT (1738035.5403648936 5704150.564802117),9.857318,POINT (1738028.590900001 5704157.555628808),0.000000
568,716.0,1,31.0,2011,POINT (1737696.7963518647 5704545.725352736),POINT (1738028.3024066975 5704171.4210171895),0.000000,POINT (1738033.000421574 5704166.11647656),7.085866,POINT (1738037.6659455353 5704160.848621506),14.122727,POINT (1738030.436974877 5704169.010870791),3.219501


In [10]:
results.index = results.TransectID.astype(int)
results

,TransectID,BaselineID,group,Year,ocean_point,linear_model_point,linear_model_distance,sqrt_model_point,sqrt_model_distance,BH_model_point,BH_model_distance,Sunamura_model_point,Sunamura_model_distance
TransectID,,,,,,,,,,,,,
2,2.0,1,0.0,2011,POINT (1734901.294958501 5698259.049972313),POINT (1735114.055735204 5697806.2951233275),0.254037,POINT (1735116.482537082 5697801.130889888),5.960058,POINT (1735120.7613459069 5697792.025586394),16.020612,POINT (1735116.1250134942 5697801.891699942),5.119430
3,3.0,1,0.0,2011,POINT (1734897.1336462596 5698251.896706724),POINT (1735123.817820887 5697806.235080629),0.000000,POINT (1735126.9420696187 5697800.092800197),6.891193,POINT (1735128.422893738 5697797.181496459),10.157464,POINT (1735123.817820887 5697806.235080629),0.000000
4,4.0,1,0.0,2011,POINT (1734900.895041738 5698249.22434408),POINT (1735133.8567471951 5697806.811817369),0.000000,POINT (1735139.1077513464 5697796.839749171),11.270102,POINT (1735140.2502180436 5697794.670115423),13.722150,POINT (1735135.1687063214 5697804.3203042215),2.815826
5,5.0,1,0.0,2011,POINT (1734907.5357417199 5698248.764666272),POINT (1735144.04181646 5697808.236714139),0.000000,POINT (1735149.7475224163 5697797.608983061),12.062493,POINT (1735151.0044365593 5697795.267792401),14.719749,POINT (1735145.845678354 5697804.876751453),3.813564
6,6.0,1,0.0,2011,POINT (1734912.0278775676 5698248.993504751),POINT (1735153.4398400048 5697811.134778228),0.000000,POINT (1735159.1144658017 5697800.8424778795),11.752992,POINT (1735160.514576642 5697798.30303972),14.652829,POINT (1735155.2488885478 5697807.853633071),3.746808
...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,713.0,1,31.0,2011,POINT (1737658.0284376887 5704476.621755098),POINT (1738025.8406172907 5704137.926906398),0.000000,POINT (1738027.7594203593 5704136.160002659),2.608401,POINT (1738035.6560237126 5704128.88852274),13.342960,POINT (1738027.6371835656 5704136.272562751),2.442233
714,714.0,1,31.0,2011,POINT (1737665.3202893392 5704491.54016579),POINT (1738027.7069854382 5704147.046504759),0.000000,POINT (1738029.1507338744 5704145.674042167),1.992000,POINT (1738035.9118355655 5704139.2467734525),11.320573,POINT (1738028.0105901905 5704146.757890652),0.418896
715,715.0,1,31.0,2011,POINT (1737676.0880781382 5704512.156475647),POINT (1738028.590900001 5704157.555628808),0.000000,POINT (1738029.9165793657 5704156.22205927),1.880381,POINT (1738035.5403648936 5704150.564802117),9.857318,POINT (1738028.590900001 5704157.555628808),0.000000


In [11]:
results["True"] = closest_to_2000.geometry
results

,TransectID,BaselineID,group,Year,ocean_point,linear_model_point,linear_model_distance,sqrt_model_point,sqrt_model_distance,BH_model_point,BH_model_distance,Sunamura_model_point,Sunamura_model_distance,True
TransectID,,,,,,,,,,,,,,
2,2.0,1,0.0,2011,POINT (1734901.294958501 5698259.049972313),POINT (1735114.055735204 5697806.2951233275),0.254037,POINT (1735116.482537082 5697801.130889888),5.960058,POINT (1735120.7613459069 5697792.025586394),16.020612,POINT (1735116.1250134942 5697801.891699942),5.119430,POINT (1735113.825 5697806.787)
3,3.0,1,0.0,2011,POINT (1734897.1336462596 5698251.896706724),POINT (1735123.817820887 5697806.235080629),0.000000,POINT (1735126.9420696187 5697800.092800197),6.891193,POINT (1735128.422893738 5697797.181496459),10.157464,POINT (1735123.817820887 5697806.235080629),0.000000,POINT (1735123.435 5697806.987)
4,4.0,1,0.0,2011,POINT (1734900.895041738 5698249.22434408),POINT (1735133.8567471951 5697806.811817369),0.000000,POINT (1735139.1077513464 5697796.839749171),11.270102,POINT (1735140.2502180436 5697794.670115423),13.722150,POINT (1735135.1687063214 5697804.3203042215),2.815826,POINT (1735133.042 5697808.360)
5,5.0,1,0.0,2011,POINT (1734907.5357417199 5698248.764666272),POINT (1735144.04181646 5697808.236714139),0.000000,POINT (1735149.7475224163 5697797.608983061),12.062493,POINT (1735151.0044365593 5697795.267792401),14.719749,POINT (1735145.845678354 5697804.876751453),3.813564,POINT (1735142.651 5697810.828)
6,6.0,1,0.0,2011,POINT (1734912.0278775676 5698248.993504751),POINT (1735153.4398400048 5697811.134778228),0.000000,POINT (1735159.1144658017 5697800.8424778795),11.752992,POINT (1735160.514576642 5697798.30303972),14.652829,POINT (1735155.2488885478 5697807.853633071),3.746808,POINT (1735151.779 5697814.147)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,713.0,1,31.0,2011,POINT (1737658.0284376887 5704476.621755098),POINT (1738025.8406172907 5704137.926906398),0.000000,POINT (1738027.7594203593 5704136.160002659),2.608401,POINT (1738035.6560237126 5704128.88852274),13.342960,POINT (1738027.6371835656 5704136.272562751),2.442233,POINT (1738031.240 5704132.954)
714,714.0,1,31.0,2011,POINT (1737665.3202893392 5704491.54016579),POINT (1738027.7069854382 5704147.046504759),0.000000,POINT (1738029.1507338744 5704145.674042167),1.992000,POINT (1738035.9118355655 5704139.2467734525),11.320573,POINT (1738028.0105901905 5704146.757890652),0.418896,POINT (1738032.589 5704142.405)
715,715.0,1,31.0,2011,POINT (1737676.0880781382 5704512.156475647),POINT (1738028.590900001 5704157.555628808),0.000000,POINT (1738029.9165793657 5704156.22205927),1.880381,POINT (1738035.5403648936 5704150.564802117),9.857318,POINT (1738028.590900001 5704157.555628808),0.000000,POINT (1738034.428 5704151.684)


In [14]:
m = closest_to_2000.explore(color="green", tiles="Esri.WorldImagery")
for model in SUPPORTED_MODELS:
  gpd.GeoDataFrame(results[f"{model}_model_distance"], geometry=results[f"{model}_model_point"], crs=2193).explore(color="blue", m=m)
m

In [13]:
for model in SUPPORTED_MODELS:
  print(model)
  print(gpd.GeoSeries(results[f"{model}_model_point"], crs=2193).distance(results["True"]).describe())

linear
count    570.000000
mean       6.000445
std        5.519057
min        0.000000
25%        1.935287
50%        4.445318
75%        8.303948
max       34.268004
dtype: float64
sqrt
count    570.000000
mean      14.502252
std       10.759456
min        0.108578
25%        6.704562
50%       12.212582
75%       19.891591
max       67.464591
dtype: float64
BH
count    570.000000
mean      17.633678
std        7.243163
min        0.108578
25%       12.919981
50%       17.515065
75%       22.099648
max       43.260254
dtype: float64
Sunamura
count    570.000000
mean       8.375430
std        6.251937
min        0.000000
25%        3.458517
50%        7.279119
75%       12.024419
max       34.268004
dtype: float64
